In [1]:
import os
#os.environ['R_HOME'] = '/home/h/hollenberg/miniconda3/envs/milopy-envv/lib/R' #path to your R installation
#os.environ['R_USER'] = '//home/h/hollenberg/miniconda3/envs/milopy-envv/lib/python3.11/site-packages/rpy2'
#home/h/hollenberg/miniconda3/envs/milopy-envv/bin/python
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging
import milopy
import milopy.core as milo
import milopy.plot as milopl
import milopy.utils

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [2]:
# path of partly preprocessed HLCA file used in this notebook:
adata_path = "/home/h/hollenberg/MaPra/datasets_seperate/galapagos_radiation.h5ad"

# add annotation when not in adata -> None if annotation already in adata
add_annotation = "/home/h/hollenberg/MaPra/datasets_seperate/galapagos_radiation_obs_annotation.csv"

sample_identifier = "SAMP_ID" 
cell_type_identifier = "annotation" 
covariate_obs = "SAMP_condition"


In [3]:
adata = sc.read(adata_path)
if add_annotation is not None:
    adata.obs = pd.read_csv(add_annotation)

In [ ]:
d = 30
k = 50

sc.pp.neighbors(adata, n_neighbors=k, n_pcs=d)

In [ ]:
milo.make_nhoods(adata, prop=0.1)
adata.obsm["nhoods"]
adata[adata.obs['nhood_ixs_refined'] != 0].obs[['nhood_ixs_refined', 'nhood_kth_distance']]

In [ ]:
nhood_size = np.array(adata.obsm["nhoods"].sum(0)).ravel()
plt.hist(nhood_size, bins=100);

In [ ]:
milo.count_nhoods(adata, sample_col=sample_identifier)
adata.uns["nhood_adata"]

In [ ]:
adata.obs["condition_continuous"] = adata.obs[covariate_obs].astype('category').cat.codes
milo.DA_nhoods(adata, design="~condition_continuous")

In [ ]:
adata.uns["nhood_adata"].obs

In [ ]:
old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = [10,5]
plt.subplot(1,2,1)
plt.hist(adata.uns["nhood_adata"].obs.PValue, bins=50);
plt.xlabel("P-Vals");
plt.subplot(1,2,2)
plt.plot(adata.uns["nhood_adata"].obs.logFC, -np.log10(adata.uns["nhood_adata"].obs.SpatialFDR), '.');
plt.xlabel("log-Fold Change");
plt.ylabel("- log10(Spatial FDR)");
plt.tight_layout()
plt.rcParams["figure.figsize"] = old_figsize

In [ ]:
milopy.utils.build_nhood_graph(adata)

In [ ]:
nhood_adata = adata.uns["nhood_adata"].copy()
nhood_adata

In [ ]:
#import milopy.utils 
plt.rcParams["figure.figsize"] = [10,10]
milopl.plot_nhood_graph(adata,
    alpha=0.01, ## SpatialFDR level (1%)
    min_size=2 ## Size of smallest dot 
                       )

## Visualize result by celltype

In [20]:
milopy.utils.annotate_nhoods(adata, anno_col=cell_type_identifier)

ValueError: Must pass 2-d input. shape=()

In [ ]:
plt.hist(adata.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
adata.uns['nhood_adata'].obs.loc[adata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
sc.pl.violin(adata.uns['nhood_adata'], "logFC", groupby="nhood_annotation", rotation=90, show=False);
plt.axhline(y=0, color='black', linestyle='--');
plt.show()